## Data Loading

In [1]:
import gzip, json
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.loads(l)

data = []

for review in parse("Software.json.gz"):
  data.append(review)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

indices = np.arange(len(data))
indices_train, indices_test = train_test_split(indices, test_size=0.1, random_state=42)
indices_train, indices_val = train_test_split(indices_train, test_size=0.1/0.9, random_state=42)

In [ ]:
data_np = np.array(data)
data_train = data_np[indices_train]
data_val = data_np[indices_val]
data_test = data_np[indices_test]

In [ ]:
y_train = [0 if ('vote' not in d) or (d['vote']==0)  else 1 for d in data_train]
y_val = [0 if ('vote' not in d) or (d['vote']==0)  else 1 for d in data_val]
y_test = [0 if ('vote' not in d) or (d['vote']==0)  else 1 for d in data_test]

## Naive Bayes

## Non-text (review) features

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# fit the ID encoder on full data set
userID_str = np.array([[d['reviewerID']] for d in data])
userID_encoder = LabelEncoder()
userID_encoder.fit(userID_str.squeeze())
userID_one_hot_encoder = OneHotEncoder()
userID_one_hot_encoder.fit(userID_str)

itemID_str = np.array([[d['asin']] for d in data])
itemID_encoder = LabelEncoder()
itemID_encoder.fit(itemID_str.squeeze())
itemID_one_hot_encoder = OneHotEncoder()
itemID_one_hot_encoder.fit(itemID_str)

OneHotEncoder()

In [ ]:
import textstat
import os
import pickle 

if os.path.isfile('readability_scores.pickle'):
    with open('readability_scores.pickle', 'rb') as handle:
        readability_scores = pickle.load(handle)
    
else:
    # Might take 2-3 mins, please be patient!
    readability_scores = {}
    for d in data:
        if 'reviewText' in d:
            readability_scores[d['reviewText']] = textstat.flesch_reading_ease(d['reviewText'])

In [ ]:
def get_meta_features(d, length=False, rating=False, readability=False, verified=False, userID=False, itemID=False):
    feature_vec = []
    
    if length:
        if 'reviewText' not in d:
            feature_vec.append(0)
        else:
            feature_vec.append(len(d['reviewText']))
            
    if rating:
        feature_vec.append(d['overall'])
    
    if readability:
        if 'reviewText' not in d:
            feature_vec.append(100)
        else:
            feature_vec.append(readability_scores[d['reviewText']])
            
    if verified:
        if 'verified' not in d:
            feature_vec.append(0)
        elif d['verified']:
            feature_vec.append(1)
        else:
            feature_vec.append(0)
            
    if userID:
        feature_vec.extend(list(userID_one_hot_encoder.transform([[d["reviewerID"]]]).toarray()[0]))
                                          
    if itemID:
        feature_vec.extend(list(itemID_one_hot_encoder.transform([[d["asin"]]]).toarray()[0]))      
        
    return feature_vec
        

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
def test_metrics(clf, X_t, y_t):
    y_pred = clf.predict(X_t)
    precision, recall, _, _ = precision_recall_fscore_support(y_t, y_pred, average='binary')
    accuracy = accuracy_score(y_t, y_pred)
    return accuracy, precision, recall, 2*precision*recall/(precision+recall)

In [ ]:
X_train = [get_meta_features(d, length=True, rating=True, readability=True, verified=True, userID=False, itemID=False) for d in data_train]
X_val = [get_meta_features(d, length=True, rating=True, readability=True, verified=True, userID=False, itemID=False) for d in data_val]
X_test = [get_meta_features(d, length=True, rating=True, readability=True, verified=True, userID=False, itemID=False) for d in data_test]
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler = scaler.fit(X_train)

In [ ]:
class EarlyStopping():
    """
    Early stopping to stop the training when the loss does not improve after
    certain epochs.
    """
    def __init__(self, patience=5, min_delta=0):
        """
        :param patience: how many epochs to wait before stopping when loss is
               not improving
        :param min_delta: minimum difference between new loss and old loss for
               new loss to be considered as an improvement
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
    def __call__(self, val_loss):
        if self.best_loss == None:
            self.best_loss = val_loss
        elif self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            # reset counter if validation loss improves
            self.counter = 0
        elif self.best_loss - val_loss < self.min_delta:
            self.counter += 1
            # print(f"INFO: Early stopping counter {self.counter} of {self.patience}")
            if self.counter >= self.patience:
                # print('INFO: Early stopping')
                self.early_stop = True

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.functional import cross_entropy
from tqdm import tqdm
from torch.autograd import Variable
from math import ceil
from torch.optim.lr_scheduler import StepLR


class MLPClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_prob=0.5, l2_reg=0.01, tol=1e-3, lr=1e-1):
        super(MLPClassifier, self).__init__()

        # Check if hidden_size is a tuple
        if isinstance(hidden_size, tuple):
            # Create a list of linear layers for each hidden size
            layers = []
            prev_size = input_size
            for size in hidden_size:
                layers.append(nn.Linear(prev_size, size))
                layers.append(nn.ReLU())
                layers.append(nn.Dropout(p=dropout_prob))
                prev_size = size
            self.hidden_layers = nn.Sequential(*layers)
        else:
            # Single hidden layer
            self.hidden_layers = nn.Sequential(
                nn.Linear(input_size, hidden_size),
                nn.ReLU(),
                nn.Dropout(p=dropout_prob)
            )

        self.fc_out = nn.Linear(prev_size, output_size)
        self.softmax = nn.Softmax(dim=1)
        self.l2_reg = l2_reg
        self.tol = tol
        self.class_weights = torch.tensor([0, 0], requires_grad=False)
        self.lr = lr

    def forward(self, x):
        # Input layer

        # Hidden layers
        for layer in self.hidden_layers:
            x = layer(x)

        # Output layer
        x = self.fc_out(x)  # No activation function for binary classification

        return self.softmax(x)
    
    def calculate_loss(self,x,y):
        device = torch.device('cuda')
        l2_reg_term = torch.nn.utils.parameters_to_vector(self.parameters()).norm(p=2)
        one_hot_encoded = torch.zeros(len(y), 2).to(device)
        one_hot_encoded.scatter_(1, y.unsqueeze(1), 1)
        y_pred = self.forward(x.float())
        return cross_entropy(y_pred, one_hot_encoded, weight=self.class_weights.to('cuda')) + self.l2_reg * l2_reg_term


    def fit(self, X_train, y_train, x_val, y_val, epochs=100, use_gpu=True):
        # Check if GPU is available and decide whether to use it
        use_gpu = use_gpu and torch.cuda.is_available()
        device = torch.device('cuda' if use_gpu else 'cpu')
        self.to(device)

        optimizer = optim.Adam(self.my_trainable_params(), lr=self.lr)
        lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='min', patience=3, factor=0.5, min_lr=1e-6)
        es = EarlyStopping(patience=10)



        print(self.class_weights)

        # x_train_tensor = torch.tensor(X_train).to(device)
        # y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

        x_train_tensor = torch.tensor(X_train) if not isinstance(X_train, torch.Tensor) else X_train
        x_train_tensor = x_train_tensor.to(device)
        
        y_train_tensor = torch.tensor(y_train, dtype=torch.long) if not isinstance(X_train, torch.Tensor) else y_train
        y_train_tensor = y_train_tensor.to(device)

        # Calculate class weights if not provided
        class_counts = torch.bincount(y_train_tensor)
        self.class_weights = 1.0 - (class_counts.float() / torch.sum(class_counts))
        self.class_weights.to(device)


        x_valid = torch.tensor(x_val).to(device)
        y_valid = torch.tensor(y_val).to(device)

        prev_loss = 1e12
        loss = 1e6

        epoch = 0
        while epoch < epochs:
            loss = self.calculate_loss(x_train_tensor, y_train_tensor)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            val_loss = self.calculate_loss(x_valid, y_valid)
            lr_scheduler.step(val_loss)
            es(val_loss)

            if es.early_stop:
                break

            epoch += 1

        print(f"Training stopped after {epoch} epochs.")

    def my_trainable_params(self):
        trainable_params = []
        for layer in self.hidden_layers:
            # print(type(layer))
            if isinstance(layer, nn.Linear):
                trainable_params.extend(layer.parameters())

        if isinstance(self.fc_out, nn.Linear):
            trainable_params.extend(self.fc_out.parameters())
        return trainable_params

    def predict(self, x, use_gpu=True):
        # Move inputs to GPU if available
        device = torch.device('cuda' if use_gpu else 'cpu')
        inputs = Variable(torch.FloatTensor(x)).to(device)
        prediction = self.forward(inputs)
        return torch.argmax(prediction.data.cpu(), 1).numpy() if use_gpu else torch.argmax(prediction.data, 1).numpy()

# Example usage:
# classifier = MLPClassifier(input_size=..., hidden_size=..., output_size=...)
# classifier.fit(X_train, y_train, epochs=..., batch_size=..., use_gpu=True)
# predictions = classifier.predict(X_test, use_gpu=True)


In [ ]:
# Logistic regression
# from sklearn.linear_model import LogisticRegression

clf = MLPClassifier(input_size=len(X_train[0]),hidden_size=(100,10,),output_size=2)
clf.fit(scaler.transform(X_train), y_train, scaler.transform(X_val), y_val, epochs=100)
test_metrics(clf, scaler.transform(X_test), y_test)

tensor([0.2783, 0.7217])
Training stopped after 70 epochs.


(0.728125544140693, 0.5074699315031552, 0.736920426065163, 0.601041234149925)

In [ ]:
np.unique(y_train, return_counts=True)

(array([0, 1]), array([265249, 102299]))

In [ ]:
# Define the hyperparameter grid
param_grid = {
    'hidden_size': [(5,), (64,), (128,), (64, 32), (64, 32, 16)],
    'l2_reg': [0.001, 0.01, 0.1, 1, 10, 100],
    # 'tol': [1e-3, 1e-4, 1e-5]
}

In [ ]:
from sklearn.model_selection import ParameterGrid



# Generate all possible combinations of hyperparameters
grid = ParameterGrid(param_grid)

scores_notext = []

# Iterate over the hyperparameter combinations
bar = tqdm(grid)
for params in bar:
    print("Testing hyperparameters:", params)

    # Create the model with the current hyperparameters
    model = MLPClassifier(input_size=len(X_train[0]), output_size=2, **params)

    # Train the model
    model.fit(scaler.transform(X_train), y_train, scaler.transform(X_val), y_val, epochs=100, use_gpu=True)
    metrics = test_metrics(model, scaler.transform(X_test), y_test)
    print(metrics)
    scores_notext.append({'params':params, 'scores':metrics})
    bar.set_postfix()


In [ ]:
# with open('mlp_notext.json', 'w') as f:
#     json.dump(scores_notext, f)

## Text Features

In [ ]:
train_documents = []
for d in data_train: 
    strs = []
    if 'summary' in d:
        strs.append(d['summary'])
    if 'reviewText' in d:
        strs.append(d['reviewText'])
    train_documents.append(" ".join(strs))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer = CountVectorizer(stop_words='english', max_features=300)
bow_vectorizer.fit(train_documents)

CountVectorizer(max_features=300, stop_words='english')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=300)
tfidf_vectorizer.fit(train_documents)

TfidfVectorizer(max_features=300, stop_words='english')

In [ ]:
w2v_features = np.load('text-feature_word2vec.npz')['word2vec']
w2v_dict = {}

count = 0 
for d in data:
    if 'reviewText' in d:
        w2v_dict[d['reviewText']] = w2v_features[count, :]
    else:
        w2v_dict[''] =  w2v_features[count, :]
    count+=1

In [ ]:
all_docs = []
for d in data: 
    strs = []
    if 'summary' in d:
        strs.append(d['summary'])
    if 'reviewText' in d:
        strs.append(d['reviewText'])
    all_docs.append(" ".join(strs))
    

In [ ]:
tfidf_vecs = tfidf_vectorizer.transform(all_docs).toarray()

tfidf_dict = {}
count = 0
for d in data:
    if 'reviewText' in d:
        tfidf_dict[d['reviewText']] = tfidf_vecs[count, :]
    else:
        tfidf_dict[''] =  tfidf_vecs[count, :]
    count+=1

In [ ]:
bow_vecs = bow_vectorizer.transform(all_docs).toarray()

bow_dict = {}
count = 0
for d in data:
    if 'reviewText' in d:
        bow_dict[d['reviewText']] = bow_vecs[count, :]
    else:
        bow_dict[''] =  bow_vecs[count, :]
    count+=1

In [ ]:
def get_text_features(d, BoW=False, tfidf=False, w2v=False):
    
    feature_vec = []
    
    if 'reviewText' in d:
        reviewText = d['reviewText']
    else:
        reviewText = ''
    
    if BoW:
        feature_vec.extend(bow_dict[reviewText].tolist())
    if tfidf:
        feature_vec.extend(tfidf_dict[reviewText].ravel())
    if w2v:
        feature_vec.extend(w2v_dict[reviewText].tolist())
        
    return feature_vec
       

In [ ]:
X_train = [get_text_features(d, BoW=True, tfidf=False, w2v=False) for d in data_train]
X_val = [get_text_features(d, BoW=True, tfidf=False, w2v=False) for d in data_val]
X_test = [get_text_features(d, BoW=True, tfidf=False, w2v=False) for d in data_test]

In [ ]:
from sklearn.model_selection import ParameterGrid

# Generate all possible combinations of hyperparameters
grid = ParameterGrid(param_grid)

scores_bow = []

# Iterate over the hyperparameter combinations
bar = tqdm(grid)
X_train = torch.tensor(X_train).to('cuda')
y_train = torch.tensor(y_train).to('cuda')
for params in bar:
    print("Testing hyperparameters:", params)

    # Create the model with the current hyperparameters
    model = MLPClassifier(input_size=len(X_train[0]), output_size=2, **params)

    # Train the model
    model.fit(X_train, y_train, X_val, y_val, epochs=100, use_gpu=True)
    metrics = test_metrics(model, X_test, y_test)
    print(metrics)
    scores_bow.append({'params':params, 'scores':metrics})
    bar.set_postfix()

with open('mlp_bow.json', 'w') as f:
    json.dump(scores_bow, f)

  0%|          | 0/30 [00:00<?, ?it/s]/tmp/ipykernel_579864/491538201.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train).to('cuda')
/tmp/ipykernel_579864/491538201.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train).to('cuda')


Testing hyperparameters: {'hidden_size': (5,), 'l2_reg': 0.001}
tensor([0, 0])
Training stopped after 73 epochs.


  3%|▎         | 1/30 [00:02<01:25,  2.95s/it]

(0.72122584015323, 0.49882117175527524, 0.662828947368421, 0.5692473262931323)
Testing hyperparameters: {'hidden_size': (5,), 'l2_reg': 0.01}
tensor([0, 0])
Training stopped after 61 epochs.


  7%|▋         | 2/30 [00:05<01:18,  2.81s/it]

...

(0.7220964652620582, 0.0, 0.0, nan)
Testing hyperparameters: {'hidden_size': (64, 32, 16), 'l2_reg': 10}
tensor([0, 0])
Training stopped after 100 epochs.


 97%|█████████▋| 29/30 [01:54<00:04,  4.28s/it]

(0.2779035347379418, 0.2779035347379418, 1.0, 0.434936639869192)
Testing hyperparameters: {'hidden_size': (64, 32, 16), 'l2_reg': 100}
tensor([0, 0])
Training stopped after 100 epochs.


100%|██████████| 30/30 [01:58<00:00,  3.96s/it]

(0.2779035347379418, 0.2779035347379418, 1.0, 0.434936639869192)


In [ ]:
X_train = [get_text_features(d, BoW=False, tfidf=True, w2v=False) for d in data_train]
X_val = [get_text_features(d, BoW=False, tfidf=True, w2v=False) for d in data_val]
X_test = [get_text_features(d, BoW=False, tfidf=True, w2v=False) for d in data_test]

In [ ]:
from sklearn.model_selection import ParameterGrid

# Generate all possible combinations of hyperparameters
grid = ParameterGrid(param_grid)

scores_tfidf = []

# Iterate over the hyperparameter combinations
bar = tqdm(grid)
X_train = torch.tensor(X_train).to('cuda')
y_train = torch.tensor(y_train).to('cuda')
for params in bar:
    print("Testing hyperparameters:", params)

    # Create the model with the current hyperparameters
    model = MLPClassifier(input_size=len(X_train[0]), output_size=2, **params)

    # Train the model
    model.fit(X_train, y_train, X_val, y_val, epochs=100, use_gpu=True)
    metrics = test_metrics(model, X_test, y_test)
    print(metrics)
    scores_tfidf.append({'params':params, 'scores':metrics})
    bar.set_postfix()

with open('mlp_tfidf.json', 'w') as f:
    json.dump(scores_tfidf, f)

  0%|          | 0/30 [00:00<?, ?it/s]/tmp/ipykernel_579864/1864138003.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train).to('cuda')


Testing hyperparameters: {'hidden_size': (5,), 'l2_reg': 0.001}
tensor([0, 0])
Training stopped after 63 epochs.


  3%|▎         | 1/30 [00:28<13:43, 28.39s/it]

(0.7146743862092982, 0.49030203060121724, 0.675125313283208, 0.5680582556262151)
Testing hyperparameters: {'hidden_size': (5,), 'l2_reg': 0.01}
tensor([0, 0])
Training stopped after 53 epochs.


  7%|▋         | 2/30 [00:32<06:40, 14.32s/it]

...

(0.2779035347379418, 0.2779035347379418, 1.0, 0.434936639869192)
Testing hyperparameters: {'hidden_size': (64, 32, 16), 'l2_reg': 10}
tensor([0, 0])
Training stopped after 100 epochs.


 97%|█████████▋| 29/30 [02:57<00:05,  5.63s/it]

(0.2779035347379418, 0.2779035347379418, 1.0, 0.434936639869192)
Testing hyperparameters: {'hidden_size': (64, 32, 16), 'l2_reg': 100}
tensor([0, 0])
Training stopped after 100 epochs.


100%|██████████| 30/30 [03:03<00:00,  6.12s/it]

(0.2779035347379418, 0.2779035347379418, 1.0, 0.434936639869192)


In [ ]:
X_train = [get_text_features(d, BoW=False, tfidf=False, w2v=True) for d in data_train]
X_val = [get_text_features(d, BoW=False, tfidf=False, w2v=True) for d in data_val]
X_test = [get_text_features(d, BoW=False, tfidf=False, w2v=True) for d in data_test]

# Generate all possible combinations of hyperparameters
grid = ParameterGrid(param_grid)

scores_w2v = []

# Iterate over the hyperparameter combinations
bar = tqdm(grid)
X_train = torch.tensor(X_train).to('cuda')
y_train = torch.tensor(y_train).to('cuda')
for params in bar:
    print("Testing hyperparameters:", params)

    # Create the model with the current hyperparameters
    model = MLPClassifier(input_size=len(X_train[0]), output_size=2, **params)

    # Train the model
    model.fit(X_train, y_train, X_val, y_val, epochs=100, use_gpu=True)
    metrics = test_metrics(model, X_test, y_test)
    print(metrics)
    scores_w2v.append({'params':params, 'scores':metrics})
    bar.set_postfix()

with open('mlp_w2v.json', 'w') as f:
    json.dump(scores_w2v, f)

  0%|          | 0/30 [00:00<?, ?it/s]/tmp/ipykernel_579864/4230160924.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train).to('cuda')


Testing hyperparameters: {'hidden_size': (5,), 'l2_reg': 0.001}
tensor([0, 0])
Training stopped after 97 epochs.


  3%|▎         | 1/30 [00:13<06:19, 13.09s/it]

(0.6301149225143653, 0.4169679346117573, 0.831062030075188, 0.5553171446514549)
Testing hyperparameters: {'hidden_size': (5,), 'l2_reg': 0.01}
tensor([0, 0])
Training stopped after 100 epochs.


/usr/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel_579864/2563740004.py:7: RuntimeWarning: invalid value encountered in scalar divide
  return accuracy, precision, recall, 2*precision*recall/(precision+recall)
  7%|▋         | 2/30 [00:15<03:15,  6.96s/it]

...

(0.7220964652620582, 0.0, 0.0, nan)
Testing hyperparameters: {'hidden_size': (64, 32, 16), 'l2_reg': 10}
tensor([0, 0])
Training stopped after 100 epochs.


 97%|█████████▋| 29/30 [01:43<00:03,  3.37s/it]

(0.2779035347379418, 0.2779035347379418, 1.0, 0.434936639869192)
Testing hyperparameters: {'hidden_size': (64, 32, 16), 'l2_reg': 100}
tensor([0, 0])
Training stopped after 100 epochs.


100%|██████████| 30/30 [01:46<00:00,  3.55s/it]

(0.2779035347379418, 0.2779035347379418, 1.0, 0.434936639869192)


## Text + HandCrafted Features

In [ ]:
def get_features(d, BoW=False, tfidf=False, w2v=False):
    a = get_text_features(d, BoW, tfidf, w2v)
    a.extend(get_meta_features(d, True, True, True, True))
    return a

In [ ]:
X_train = [get_features(d, BoW=True, tfidf=False, w2v=False) for d in data_train]
X_val = [get_features(d, BoW=True, tfidf=False, w2v=False) for d in data_val]
X_test = [get_features(d, BoW=True, tfidf=False, w2v=False) for d in data_test]

# Generate all possible combinations of hyperparameters
grid = ParameterGrid(param_grid)

scores_bow_hand = []

# Iterate over the hyperparameter combinations
bar = tqdm(grid)
X_train = torch.tensor(X_train).to('cuda')
y_train = torch.tensor(y_train).to('cuda')
for params in bar:
    print("Testing hyperparameters:", params)

    # Create the model with the current hyperparameters
    model = MLPClassifier(input_size=len(X_train[0]), output_size=2, **params)

    # Train the model
    model.fit(X_train, y_train, X_val, y_val, epochs=100, use_gpu=True)
    metrics = test_metrics(model, X_test, y_test)
    print(metrics)
    scores_bow_hand.append({'params':params, 'scores':metrics})
    bar.set_postfix()

with open('mlp_bow_hand.json', 'w') as f:
    json.dump(scores_bow_hand, f)

  0%|          | 0/30 [00:00<?, ?it/s]/tmp/ipykernel_579864/1665296896.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train).to('cuda')


Testing hyperparameters: {'hidden_size': (5,), 'l2_reg': 0.001}
<class 'torch.nn.modules.linear.Linear'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.dropout.Dropout'>
tensor([0, 0])
Training stopped after 55 epochs.


  3%|▎         | 1/30 [00:13<06:43, 13.90s/it]

(0.5060508445063555, 0.31536458333333334, 0.6639254385964912, 0.4276129943502825)
Testing hyperparameters: {'hidden_size': (5,), 'l2_reg': 0.01}
<class 'torch.nn.modules.linear.Linear'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.dropout.Dropout'>
tensor([0, 0])
Training stopped after 100 epochs.


  7%|▋         | 2/30 [00:16<03:28,  7.46s/it]

...

(0.2779035347379418, 0.2779035347379418, 1.0, 0.434936639869192)
Testing hyperparameters: {'hidden_size': (64, 32, 16), 'l2_reg': 10}
<class 'torch.nn.modules.linear.Linear'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.dropout.Dropout'>
<class 'torch.nn.modules.linear.Linear'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.dropout.Dropout'>
<class 'torch.nn.modules.linear.Linear'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.dropout.Dropout'>
tensor([0, 0])
Training stopped after 100 epochs.


/usr/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel_579864/2563740004.py:7: RuntimeWarning: invalid value encountered in scalar divide
  return accuracy, precision, recall, 2*precision*recall/(precision+recall)
 97%|█████████▋| 29/30 [01:52<00:04,  4.02s/it]

(0.7220964652620582, 0.0, 0.0, nan)
Testing hyperparameters: {'hidden_size': (64, 32, 16), 'l2_reg': 100}
<class 'torch.nn.modules.linear.Linear'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.dropout.Dropout'>
<class 'torch.nn.modules.linear.Linear'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.dropout.Dropout'>
<class 'torch.nn.modules.linear.Linear'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.dropout.Dropout'>
tensor([0, 0])
Training stopped after 100 epochs.


100%|██████████| 30/30 [01:57<00:00,  3.90s/it]

(0.2779035347379418, 0.2779035347379418, 1.0, 0.434936639869192)


In [ ]:
X_train = [get_features(d, BoW=False, tfidf=True, w2v=False) for d in data_train]
X_val = [get_features(d, BoW=False, tfidf=True, w2v=False) for d in data_val]
X_test = [get_features(d, BoW=False, tfidf=True, w2v=False) for d in data_test]

# Generate all possible combinations of hyperparameters
grid = ParameterGrid(param_grid)

scores_tfidf_hand = []

# Iterate over the hyperparameter combinations
bar = tqdm(grid)
X_train = torch.tensor(X_train).to('cuda')
y_train = torch.tensor(y_train).to('cuda')
for params in bar:
    print("Testing hyperparameters:", params)

    # Create the model with the current hyperparameters
    model = MLPClassifier(input_size=len(X_train[0]), output_size=2, **params)

    # Train the model
    model.fit(X_train, y_train, X_val, y_val, epochs=100, use_gpu=True)
    metrics = test_metrics(model, X_test, y_test)
    print(metrics)
    scores_tfidf_hand.append({'params':params, 'scores':metrics})
    bar.set_postfix()

with open('mlp_tfidf_hand.json', 'w') as f:
    json.dump(scores_tfidf_hand, f)

  0%|          | 0/30 [00:00<?, ?it/s]/tmp/ipykernel_579864/940750581.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train).to('cuda')


Testing hyperparameters: {'hidden_size': (5,), 'l2_reg': 0.001}
<class 'torch.nn.modules.linear.Linear'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.dropout.Dropout'>
tensor([0, 0])
Training stopped after 17 epochs.


/usr/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel_579864/2563740004.py:7: RuntimeWarning: invalid value encountered in scalar divide
  return accuracy, precision, recall, 2*precision*recall/(precision+recall)
  3%|▎         | 1/30 [00:29<14:06, 29.19s/it]

(0.7220964652620582, 0.0, 0.0, nan)
Testing hyperparameters: {'hidden_size': (5,), 'l2_reg': 0.01}
<class 'torch.nn.modules.linear.Linear'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.dropout.Dropout'>
tensor([0, 0])
Training stopped after 61 epochs.


  7%|▋         | 2/30 [00:33<06:55, 14.84s/it]

...

(0.2779035347379418, 0.2779035347379418, 1.0, 0.434936639869192)
Testing hyperparameters: {'hidden_size': (64, 32, 16), 'l2_reg': 10}
<class 'torch.nn.modules.linear.Linear'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.dropout.Dropout'>
<class 'torch.nn.modules.linear.Linear'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.dropout.Dropout'>
<class 'torch.nn.modules.linear.Linear'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.dropout.Dropout'>
tensor([0, 0])
Training stopped after 100 epochs.


 97%|█████████▋| 29/30 [03:01<00:06,  6.11s/it]

(0.2779035347379418, 0.2779035347379418, 1.0, 0.434936639869192)
Testing hyperparameters: {'hidden_size': (64, 32, 16), 'l2_reg': 100}
<class 'torch.nn.modules.linear.Linear'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.dropout.Dropout'>
<class 'torch.nn.modules.linear.Linear'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.dropout.Dropout'>
<class 'torch.nn.modules.linear.Linear'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.dropout.Dropout'>
tensor([0, 0])
Training stopped after 100 epochs.


/usr/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel_579864/2563740004.py:7: RuntimeWarning: invalid value encountered in scalar divide
  return accuracy, precision, recall, 2*precision*recall/(precision+recall)
100%|██████████| 30/30 [03:07<00:00,  6.27s/it]

(0.7220964652620582, 0.0, 0.0, nan)


In [ ]:
X_train = [get_features(d, BoW=False, tfidf=False, w2v=True) for d in data_train]
X_val = [get_features(d, BoW=False, tfidf=False, w2v=True) for d in data_val]
X_test = [get_features(d, BoW=False, tfidf=False, w2v=True) for d in data_test]

# Generate all possible combinations of hyperparameters
grid = ParameterGrid(param_grid)

scores_w2v_hand = []

# Iterate over the hyperparameter combinations
bar = tqdm(grid)
X_train = torch.tensor(X_train).to('cuda')
y_train = torch.tensor(y_train).to('cuda')
for params in bar:
    print("Testing hyperparameters:", params)

    # Create the model with the current hyperparameters
    model = MLPClassifier(input_size=len(X_train[0]), output_size=2, **params)

    # Train the model
    model.fit(X_train, y_train, X_val, y_val, epochs=100, use_gpu=True)
    metrics = test_metrics(model, X_test, y_test)
    print(metrics)
    scores_w2v_hand.append({'params':params, 'scores':metrics})
    bar.set_postfix()

with open('mlp_w2v_hand.json', 'w') as f:
    json.dump(scores_w2v_hand, f)

: 